In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# Example training data
input_texts = [...]  # List of input texts
target_texts = [...]  # List of corresponding target texts

# Tokenize the input and target texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_texts + target_texts)
input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

# Create the vocabulary mapping
word_index = tokenizer.word_index
num_tokens = len(word_index) + 1  # Add 1 for the padding token

# Pad the sequences to a fixed length
max_seq_length = ...  # Maximum sequence length
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_length, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_seq_length, padding='post')

# Prepare the input and target data
encoder_input_data = input_sequences
decoder_input_data = target_sequences[:, :-1]  # Remove the last token from target_sequences
decoder_target_data = to_categorical(target_sequences[:, 1:], num_tokens)  # Shift target_sequences by 1 and one-hot encode


In [ ]:
num_encoder_tokens = ...  # Number of unique input tokens
num_decoder_tokens = ...  # Number of unique output tokens
latent_dim = ...  # Dimensionality of the latent space (encoder/decoder hidden units)


In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


In [ ]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=validation_split)


In [ ]:
model.save('model_file.h5')
tokenizer.save('tokenizer_file.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)


In [ ]:
def encode_input_text(input_text, tokenizer, max_seq_length):
    # Tokenize the input_text
    input_tokens = tokenizer.texts_to_sequences([input_text])[0]

    # Pad the sequence to the maximum sequence length
    padded_input_tokens = pad_sequences([input_tokens], maxlen=max_seq_length, padding='post')

    # Convert to a 2D numpy array with shape (1, max_seq_length)
    encoder_input_data = np.array(padded_input_tokens)

    return encoder_input_data

input_text = "Hello, how are you?"

# Assuming you have a tokenizer and max_seq_length defined
encoder_input_data = encode_input_text(input_text, tokenizer, max_seq_length)



In [ ]:
def decode_response(input_text):
    encoder_input_data = encode_input_text(input_text)
    states_value = encoder_model.predict(encoder_input_data)

    target_seq = ...  # Start with a sequence containing the start token
    stop_condition = False
    response = ""

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token from the output distribution
        sampled_token_index = ...  # Select the token with the highest probability
        sampled_token = ...  # Convert the token into its corresponding word

        if sampled_token != 'end_token':
            response += sampled_token + " "

        # Exit condition: either hitting max length or finding the stop token
        if sampled_token == 'end_token' or len(response.split()) > max_length:
            stop_condition = True

        # Update the target sequence (previous output) and states
        target_seq = ...  # Prepare the next token for the decoder input
        states_value = [h, c]

    return response

In [ ]:
input_text = ...  # User's input
response = decode_response(input_text)
print(response)


# fastapi

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from fastapi import FastAPI
from pydantic import BaseModel
from tensorflow.keras.preprocessing.text import Tokenizer

app = FastAPI()

# Define the input and output dimensions
num_encoder_tokens = ...
num_decoder_tokens = ...
latent_dim = ...

# Define the encoder model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder model
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the entire model (encoder + decoder)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Load the trained weights into the model
model.load_weights('path_to_weights.h5')

async def load_tokenizer():
    global tokenizer  # Declare the tokenizer as a global variable
    tokenizer = Tokenizer()
    # Load the tokenizer from file or any other method
    tokenizer.load('tokenizer_file.h5')

class ChatRequest(BaseModel):
    text: str

class ChatResponse(BaseModel):
    response: str

@app.post("/chat")
async def chat(request: ChatRequest) -> ChatResponse:
    input_text = request.text
    response_text = decode_response(input_text)
    return ChatResponse(response=response_text)

def encode_input_text(input_text, tokenizer, max_seq_length):
    # Tokenize the input_text
    input_tokens = tokenizer.texts_to_sequences([input_text])[0]

    # Pad the sequence to the maximum sequence length
    padded_input_tokens = pad_sequences([input_tokens], maxlen=max_seq_length, padding='post')

    # Convert to a 2D numpy array with shape (1, max_seq_length)
    encoder_input_data = np.array(padded_input_tokens)

    return encoder_input_data
def decode_response(input_text):
    encoder_input_data = encode_input_text(input_text)
    states_value = encoder_model.predict(encoder_input_data)

    target_seq = ...  # Start with a sequence containing the start token
    stop_condition = False
    response = ""

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token from the output distribution
        sampled_token_index = ...  # Select the token with the highest probability
        sampled_token = ...  # Convert the token into its corresponding word

        if sampled_token != 'end_token':
            response += sampled_token + " "

        # Exit condition: either hitting max length or finding the stop token
        if sampled_token == 'end_token' or len(response.split()) > max_length:
            stop_condition = True

        # Update the target sequence (previous output) and states
        target_seq = ...  # Prepare the next token for the decoder input
        states_value = [h, c]

    return response

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
